In [1]:
import re
import os
import pandas as pd
import cv2
import random
import numpy as np
from tensorflow import keras
from matplotlib import pyplot as plt
from sklearn.decomposition import PCA
from tqdm import tqdm

# Data Preparation

In [34]:
path = "data"
# path to the data 

In [35]:
#read the file name
emotions = {
    1: 'neutral',
    2: 'calm',
    3: 'happy',
    4: 'sad',
    5: 'angry',
    6: 'fear',
    7: 'disgust',
    8: 'surprise'
}

emotional_intensity = {
    1: 'normal',
    2: 'strong'
}

filenames = []
feats = []
labels = []
paths = []

for (dirpath, dirnames, fn) in os.walk(path):
    for name in fn:
        filename = name.split('.')[0]
        feat = filename.split('-')[2:]
        label = feat[0]
        filenames.append(filename)
        feats.append(feat)
        labels.append(label)
        paths.append(dirpath + '/' + filename)
        
filenames[:5]

['01-01-01-01-01-01-01',
 '01-01-01-01-01-02-01',
 '01-01-01-01-02-01-01',
 '01-01-01-01-02-02-01',
 '01-01-02-01-01-01-01']

## Data Exploration

In [36]:
data = pd.DataFrame(feats, columns = ['emotion','emotional_intensity', 'statement', 'repetition', 'actor']).astype(int)

data['index'] = filenames
data.set_index('index', inplace=True)

data['emotion'] = data['emotion'].map(emotions)
data['emotional_intensity'] = data['emotional_intensity'].map(emotional_intensity)



In [37]:
data

,emotion,emotional_intensity,statement,repetition,actor
index,,,,,
01-01-01-01-01-01-01,neutral,normal,1,1,1
01-01-01-01-01-02-01,neutral,normal,1,2,1
01-01-01-01-02-01-01,neutral,normal,2,1,1
01-01-01-01-02-02-01,neutral,normal,2,2,1
01-01-02-01-01-01-01,calm,normal,1,1,1
...,...,...,...,...,...
02-01-08-01-02-02-24,surprise,normal,2,2,24
02-01-08-02-01-01-24,surprise,strong,1,1,24
02-01-08-02-01-02-24,surprise,strong,1,2,24


## Frames Extraction

To process the frames from the video, we will follow these steps:

* We start counting frames from the beginning of the video.
* When we reach the 21st frame, we select it as the first frame to include.
* After that, we skip the next two frames.
* Then, we select the frame that follows the skipped frames, which will be the 24th frame.
* We repeat this process for the rest of the video, selecting one frame every three frames.

For each selected frame, we will resize it proportionally so that the height becomes 224 pixels. The width will be adjusted automatically to maintain the original aspect ratio of the frame.

Finally, we will save each selected frame as a PNG image file.

In [46]:
def prepare_all_videos(filenames, paths, skip=1):
    nframes_tot = 0
    
    for count, video in enumerate(zip(filenames, paths)):
        # Gather all its frames
        save_frames(video[0], video[1], video[1].replace('data', 'data_frames'), skip)
        print(f"Processed videos {count+1}/{len(paths)}")
    return


def save_frames(filename, input_path, output_path, skip):
    # Initialize video reader
    cap = cv2.VideoCapture(input_path + '.mp4')
    frames = []
    count = 0
    
    if not os.path.exists(output_path):
        os.makedirs(output_path)
    try:
        # Loop through all frames
        while True:
            # Capture frame
            ret, frame = cap.read()
            if (count % skip == 0 and count > 20):
                #print(frame.shape)
                if not ret:
                    break
                frame = cv2.resize(frame, (398, 224))
                cv2.imwrite(output_path + '/' + f'{filename}_{count}' + '.png', frame)
            count += 1
    finally:
        cap.release()
    return

In [47]:
prepare_all_videos(filenames, paths, skip=3)


Processed videos 1/2874
Processed videos 2/2874
Processed videos 3/2874
Processed videos 4/2874
Processed videos 5/2874
Processed videos 6/2874
Processed videos 7/2874
Processed videos 8/2874
Processed videos 9/2874
Processed videos 10/2874
Processed videos 11/2874
Processed videos 12/2874
Processed videos 13/2874
Processed videos 14/2874
Processed videos 15/2874
Processed videos 16/2874
Processed videos 17/2874
Processed videos 18/2874
Processed videos 19/2874
Processed videos 20/2874
Processed videos 21/2874
Processed videos 22/2874
Processed videos 23/2874
Processed videos 24/2874
Processed videos 25/2874
Processed videos 26/2874
Processed videos 27/2874
Processed videos 28/2874
Processed videos 29/2874
Processed videos 30/2874
Processed videos 31/2874
Processed videos 32/2874
Processed videos 33/2874
Processed videos 34/2874
Processed videos 35/2874
Processed videos 36/2874
Processed videos 37/2874
Processed videos 38/2874
Processed videos 39/2874
Processed videos 40/2874
Processed